In [1]:
import cv2
import numpy as np
import time
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import pickle
from keras.models import load_model
import json
from mtcnn.mtcnn import MTCNN

In [9]:
def extract_face_embedding(face_image, model):
    # Convert face image to grayscale
    face_image_gray = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
    # Resize to match the expected input size of the model
    face_image_gray_resized = cv2.resize(face_image_gray, (50, 50))
    # Expand dimensions to match the input shape of the model
    face_image_gray_resized = np.expand_dims(face_image_gray_resized, axis=-1)
    face_image_gray_resized = np.expand_dims(face_image_gray_resized, axis=0)
    # Normalize pixel values
    face_image_gray_resized = face_image_gray_resized.astype('float32') / 255.0
    # Predict embedding using the model
    embedding = model.predict(face_image_gray_resized)
    return embedding.flatten().tolist()

In [2]:
detector = MTCNN()

In [8]:
def extract_face_mtcnn(image):
    # Kiểm tra xem hình ảnh có hợp lệ không
    if image is None:
        return None, None
    
    # Phát hiện khuôn mặt
    results = detector.detect_faces(image)
    
    if len(results) == 0:
        return None, None
    
    # Trả về khuôn mặt đầu tiên được phát hiện
    for result in results:
        x, y, w, h = result['box']
        face = image[y:y+h, x:x+w]
        face = cv2.resize(face, (160, 160))
        return face, (x, y, w, h)
    
    return None, None

In [4]:
def load_train_data_and_extract_embeddings(train_data_dir, model):
    faces = []
    embeddings = []
    labels = []

    for person_name in os.listdir(train_data_dir):
        person_dir = os.path.join(train_data_dir, person_name)
        if not os.path.isdir(person_dir):
            continue
        
        for filename in os.listdir(person_dir):
            image_path = os.path.join(person_dir, filename)
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Image at {image_path} could not be loaded.")
                continue

            face = extract_face_mtcnn(image)
            if face is None:
                print(f"Error: No face detected in image {image_path}.")
                continue
            face_embedding = extract_face_embedding(face, model)

            # faces.append(image)
            # embeddings.append(face_embedding)
            # labels.append(person_name)

            user_data = {
                'user_name': person_name,
                'embedding': face_embedding
            }

            # Save the user data to a JSON file
            users_db_file = 'registered_users.json'
            if os.path.exists(users_db_file):
                with open(users_db_file, 'r') as file:
                    users_db = json.load(file)
            else:
                users_db = []

            users_db.append(user_data)
            
            with open(users_db_file, 'w') as file:
                json.dump(users_db, file, indent=4)
    

In [10]:
# Đường dẫn tới thư mục chứa dữ liệu huấn luyện
train_data_dir = '../../../dataset/data/'

# Load pre-trained FaceNet model
faceNet_model = load_model('../model/model-cnn-facerecognition.h5')

# Load training data and extract embeddings
load_train_data_and_extract_embeddings(train_data_dir, faceNet_model)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numerical tuple
>  - Expected Ptr<cv::UMat> for argument 'src'


In [11]:
# Load the registered user data from JSON file
def load_registered_users():
    users_db_file = 'registered_users.json'
    if os.path.exists(users_db_file):
        with open(users_db_file, 'r') as file:
            users_db = json.load(file)
        return users_db
    return []

registered_users = load_registered_users()

In [12]:
# Hàm để phát hiện và nhận diện khuôn mặt từ camera
def recognize_faces_from_camera(model, registered_users):
    # Khởi tạo video stream từ camera
    vs = cv2.VideoCapture(0)
    time.sleep(2.0)

    while True:
        # Đọc frame từ video stream
        ret, frame = vs.read()
        if not ret:
            break

        # Chuyển đổi frame sang ảnh grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Sử dụng bộ phát hiện khuôn mặt Cascade Classifier của OpenCV
        face_cascade = cv2.CascadeClassifier('../utils/haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            # Phát hiện được khuôn mặt, cắt ảnh khuôn mặt và trích xuất embedding
            face_image = frame[y:y+h, x:x+w]
            face_embedding = extract_face_embedding(face_image, model)

            # Tìm người dùng có embedding gần nhất
            min_dist = float('inf')
            recognized_user = None
            for user in registered_users:
                user_embedding = np.array(user['embedding'])
                dist = np.linalg.norm(user_embedding - face_embedding)
                if dist < min_dist:
                    min_dist = dist
                    recognized_user = user

            # Vẽ hộp giới hạn và hiển thị kết quả dự đoán trên frame
            if recognized_user is not None:
                name = recognized_user['user_name']
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                text = f'{name}: {min_dist:.2f}'
                cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
            else:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                cv2.putText(frame, "Unknown", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

        # Hiển thị frame
        cv2.imshow("Face Recognition", frame)

        # Thoát khỏi vòng lặp nếu nhấn phím 'q'
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # Giải phóng video stream và đóng cửa sổ hiển thị
    vs.release()
    cv2.destroyAllWindows()

In [13]:
recognize_faces_from_camera(faceNet_model, registered_users)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━